---
title: Migrating from RefineDocumentsChain
---

In [ ]:
%pip install --upgrade --quiet langchain-openai

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [2]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book"}),
]

In [5]:
from langchain.chains import LLMChain, RefineDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
# The prompt here should take as an input variable the
# `document_variable_name`
summarize_prompt = ChatPromptTemplate(
    [
        ("human", "Write a concise summary of the following: {context}"),
    ]
)
initial_llm_chain = LLMChain(llm=llm, prompt=summarize_prompt)
initial_response_name = "existing_answer"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_llm_chain = LLMChain(llm=llm, prompt=refine_prompt)
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)

In [6]:
result = chain.invoke(documents)
result["output_text"]

'Apples are typically red in color, blueberries are blue, and bananas are yellow.'

In [7]:
from typing import List

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

summarize_prompt = ChatPromptTemplate(
    [
        ("human", "Write a concise summary of the following: {context}"),
    ]
)

initial_summary = (
    RunnableLambda(lambda docs: docs[0].page_content)  # select first document
    | summarize_prompt  # Format content into summarize prompt
    | llm  # Generate response
    | StrOutputParser()  # Cast to string
)

refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])


def _refine_remaining_docs(input_dict: dict) -> RunnableSequence:
    """Create a sequence of summarizers, each adding context from a document."""
    steps = []
    for document in input_dict["documents"]:
        steps.append(
            # format text from document[i] into refine prompt
            refine_prompt.partial(context=document.page_content)
            | llm  # generate response
            | StrOutputParser()  # cast to string
        )
    return RunnableSequence(*steps)


chain = (
    {
        # create initial summary
        "existing_answer": initial_summary,
        # pass along remaining documents
        "documents": RunnableLambda(lambda docs: docs[1:]),
    }
    # create and run a
    | RunnableLambda(_refine_remaining_docs)
)

In [8]:
chain.invoke(documents)

'Apples are a red fruit, blueberries are blue, and bananas are yellow.'